In [12]:
import os 
import pathlib
from xml.etree import ElementTree

import dask
import dask.array as da
from matplotlib import pyplot as plt

import SessionTools as st 
import SessionTools.two_photon

%load_ext autoreload
%autoreload 2 

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
basedir = pathlib.PurePath('/media/mplitt/SSD_storage/2PData/12_16_2022/EPG_GCaMP7f_CsChrimsontdT_fly1/fixed_cw_remapping-000/')
session = "fixed_cw_remapping-000"


basename_input = pathlib.Path(basedir.joinpath(session)).with_suffix('.xml')
fname_xml = basename_input.with_suffix('.xml')
fname_vr_xml = pathlib.Path(str(basename_input) + '_Cycle00001_VoltageRecording_001').with_suffix('.xml')
fname_mp_xml = pathlib.Path(str(basename_input) + '_Cycle00001_MarkPoints').with_suffix('.xml')

mdata_root = ElementTree.parse(fname_xml).getroot()
    
class MetadataError(Exception):
    """Error while extracting metadata."""
# st.two_photon.metadata.read(basename_input,pathlib.Path('.'))

In [15]:
def state_value(key, type_fn=str):
    element = mdata_root.find(f'.//PVStateValue[@key="{key}"]')
    value = element.attrib['value']
    return type_fn(value)

def indexed_value(key, index, type_fn=None, required=True):
    element = mdata_root.find(f'.//PVStateValue[@key="{key}"]/IndexedValue[@index="{index}"]')
    if element is None:
        if required:
            raise MetadataError('Could not find required key:index of %s:%s' % (key, index))
        return None
    value = element.attrib['value']
    return type_fn(value)

In [17]:
sequences = mdata_root.findall('Sequence')
num_sequences = len(sequences)

num_frames_per_sequence = len(sequences[0].findall('Frame'))

if num_sequences == 1:
    num_frames = num_frames_per_sequence
    num_z_planes = 1
else:
    # If the last sequence has a different number of frames, ignore it.
    num_frames_last_sequence = len(sequences[-1].findall('Frame'))
    if num_frames_per_sequence != num_frames_last_sequence:
        logging.warning('Skipping final stack because it was found with fewer z-planes (%d, expected: %d).',
                        num_frames_last_sequence, num_frames_per_sequence)
        num_sequences -= 1
    num_frames = num_sequences
    num_z_planes = num_frames_per_sequence


In [19]:
num_channels = len(mdata_root.find('Sequence/Frame').findall('File'))
num_y_px = state_value('linesPerFrame', int)
num_x_px = state_value('pixelsPerLine', int)

In [29]:
laser_power = indexed_value('laserPower', '0', float, required=False)

In [31]:
key, index = 'laserPower', 0
element = mdata_root.find(f'.//PVStateValue[@key="{key}"]/IndexedValue[@index="{index}"]')

In [38]:
element

<Element 'IndexedValue' at 0x7fd1ffd53ec0>

In [36]:
float(element.attrib['value'])

229.98046875

In [30]:
laser_power